# Notebook Imports

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import date
import seaborn as sns
from scipy import stats

import matplotlib.pyplot as plt
import warnings

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.manifold import TSNE

warnings.filterwarnings('ignore')

# Constants

In [2]:
FILE = '/kaggle/input/data-science-for-good/Dept_49-00035/49-00035_Incidents_2016.csv'

# Load Data

In [3]:
data = pd.read_csv(FILE, low_memory=False)
df = pd.DataFrame(data)

df.drop([0], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

df.head()

,CRIME_TYPE,INCIDENT_DATE,INCIDENT_UNIQUE_IDENTIFIER,INCIDENT_REASON,LOCATION_DISTRICT,LOCATION_FULL_STREET_ADDRESS_OR_INTERSECTION,LOCATION_CITY,LOCATION_STATE
0,DOMESTIC VIOLENCE,12/6/16 16:12,16-064496,THREAT CRIME:INT:TERRORIZE,04X,1700 MARTIN LUTHER KING JR W,Oakland,CA
1,STOLEN VEHICLE,12/6/16 7:20,16-064497,VEHICLE THEFT - AUTO,27X,1600 HIGH ST,Oakland,CA
2,FELONY ASSAULT,12/6/16 19:00,16-064499,OBSTRUCT/RESIST/ETC PUBLIC/PEACE OFFICER/EMERG...,08X,2800 TELEGRAPH AV,Oakland,CA
3,NARCOTICS,12/6/16 19:20,16-064501,POSSESS CONTROLLED SUBSTANCE PARAPHERNALIA,07X,34TH ST & LINDEN ST,Oakland,CA
4,DISORDERLY CONDUCT,12/6/16 18:48,16-064505,DISORDERLY CONDUCT - ALCOHOL,04X,2100 SAN PABLO AV,Oakland,CA


# Step 1: Data Exploration and Preprocessing

In [4]:
print(f"\nThe shape of the data: {df.shape}\n")
print(f"The types of the data: \n{df.dtypes}\n")
print(f"The empty row in the data: \n{df.isnull().sum()}")


The shape of the data: (10769, 8)

The types of the data: 
CRIME_TYPE                                      object
INCIDENT_DATE                                   object
INCIDENT_UNIQUE_IDENTIFIER                      object
INCIDENT_REASON                                 object
LOCATION_DISTRICT                               object
LOCATION_FULL_STREET_ADDRESS_OR_INTERSECTION    object
LOCATION_CITY                                   object
LOCATION_STATE                                  object
dtype: object

The empty row in the data: 
CRIME_TYPE                                      709
INCIDENT_DATE                                     2
INCIDENT_UNIQUE_IDENTIFIER                        0
INCIDENT_REASON                                   0
LOCATION_DISTRICT                               197
LOCATION_FULL_STREET_ADDRESS_OR_INTERSECTION      0
LOCATION_CITY                                     0
LOCATION_STATE                                    0
dtype: int64


### Handling missing values in the dataframe

In [5]:
# Extracting dates

df['INCIDENT_DATE'] = pd.to_datetime(df.INCIDENT_DATE, format="mixed")
df["INCIDENT_YEAR"] = df["INCIDENT_DATE"].dt.strftime('%Y')
df["INCIDENT_MONTH"] = df["INCIDENT_DATE"].dt.strftime('%m')
df["INCIDENT_WEEKDAY"] = df["INCIDENT_DATE"].dt.strftime('%w')
df["INCIDENT_TIME"] = df["INCIDENT_DATE"].dt.strftime('%X')
df['INCIDENT_DATE'] = df['INCIDENT_DATE'].astype(str)
df["INCIDENT_DATE"] = df["INCIDENT_DATE"].str.split(" ").str[0].str.split("-").str[2]  # Corrected date splitting

In [6]:
df.head()

,CRIME_TYPE,INCIDENT_DATE,INCIDENT_UNIQUE_IDENTIFIER,INCIDENT_REASON,LOCATION_DISTRICT,LOCATION_FULL_STREET_ADDRESS_OR_INTERSECTION,LOCATION_CITY,LOCATION_STATE,INCIDENT_YEAR,INCIDENT_MONTH,INCIDENT_WEEKDAY,INCIDENT_TIME
0,DOMESTIC VIOLENCE,06,16-064496,THREAT CRIME:INT:TERRORIZE,04X,1700 MARTIN LUTHER KING JR W,Oakland,CA,2016,12,2,16:12:00
1,STOLEN VEHICLE,06,16-064497,VEHICLE THEFT - AUTO,27X,1600 HIGH ST,Oakland,CA,2016,12,2,07:20:00
2,FELONY ASSAULT,06,16-064499,OBSTRUCT/RESIST/ETC PUBLIC/PEACE OFFICER/EMERG...,08X,2800 TELEGRAPH AV,Oakland,CA,2016,12,2,19:00:00
3,NARCOTICS,06,16-064501,POSSESS CONTROLLED SUBSTANCE PARAPHERNALIA,07X,34TH ST & LINDEN ST,Oakland,CA,2016,12,2,19:20:00
4,DISORDERLY CONDUCT,06,16-064505,DISORDERLY CONDUCT - ALCOHOL,04X,2100 SAN PABLO AV,Oakland,CA,2016,12,2,18:48:00


In [7]:
df['INCIDENT_YEAR'].replace(to_replace=np.nan, value=0, inplace=True)
df['INCIDENT_YEAR'] = df['INCIDENT_YEAR'].astype(int)
replace = df[(df['INCIDENT_YEAR'] < 2016) | (df['INCIDENT_YEAR'] > 2017)]['INCIDENT_YEAR'].tolist()
value = round(df[(df['INCIDENT_YEAR'] >= 2016) | (df['INCIDENT_YEAR'] <= 2017)]['INCIDENT_YEAR'].mean())
df['INCIDENT_YEAR'].replace(to_replace=replace, value=value, inplace=True)

In [8]:
for feature in df.columns:
    df[feature].replace(to_replace=np.nan, value="UNKNOWN", inplace=True) 

In [9]:
print(f"The empty row in the data: \n{df.isnull().sum()}")

The empty row in the data: 
CRIME_TYPE                                      0
INCIDENT_DATE                                   0
INCIDENT_UNIQUE_IDENTIFIER                      0
INCIDENT_REASON                                 0
LOCATION_DISTRICT                               0
LOCATION_FULL_STREET_ADDRESS_OR_INTERSECTION    0
LOCATION_CITY                                   0
LOCATION_STATE                                  0
INCIDENT_YEAR                                   0
INCIDENT_MONTH                                  0
INCIDENT_WEEKDAY                                0
INCIDENT_TIME                                   0
dtype: int64


In [10]:
df.shape

(10769, 12)

In [11]:
df['CRIME_TYPE'].value_counts()

CRIME_TYPE
BURG - AUTO                           2052
STOLEN VEHICLE                        1683
VANDALISM                              877
PETTY THEFT                            853
MISDEMEANOR ASSAULT                    780
UNKNOWN                                709
DOMESTIC VIOLENCE                      674
ROBBERY                                654
BURG - RESIDENTIAL                     489
FELONY ASSAULT                         284
GRAND THEFT                            270
NARCOTICS                              215
RECOVERED O/S STOLEN                   195
DISORDERLY CONDUCT                     164
FRAUD                                  149
BURG - COMMERCIAL                      145
WEAPONS                                 91
PROSTITUTION                            77
OTHER                                   71
THREATS                                 70
DUI                                     62
RECOVERED VEHICLE - OAKLAND STOLEN      35
ARSON                                   29
